In [5]:
import urllib.request
import re
import pandas as pd
import datetime
from pandas.tseries.offsets import BDay

In [6]:
def get_support_resistance(symbol):
    url = 'http://www.stockta.com/cgi-bin/analysis.pl?symb={0}&cobrand=&mode=stock'.format(symbol)
    
    support = (0, 0)
    resistance = (0, 0)
    with urllib.request.urlopen(url) as f:
        contend = f.read().decode('iso-8859-1')
        price = float((re.findall(r'Close=([\d.]+) Volume=', contend))[0])
        
        print(price)
        match = re.search(r'&support=(.*)&', contend)
        if match:
            levels_str = re.findall(r'([\d.]+),([\d.]+)', match.group())
            levels_num = [(float(t[0]), int(t[1])) for t in levels_str]
            supports = [x for x in levels_num if x[0] < price and x[1] >= 5 ]
            if supports:                
                support =  sorted(supports).pop()
            resistances = [x for x in levels_num if x[0] > price and x[1] >=5]
            if resistances:
                resistance = sorted(resistances)[0]
            
    return price, support, resistance

In [7]:
def calculate_atr(sym, period=10):
    
    high = df_train['High'].values
    low = df_train['Low'].values
    close = df_train['Close'].values
    
    atr = ta.ATR(high, low, close, period)[-1]
    return atr

In [8]:
f = open('sp500-tickers.txt', 'r')
symbols = f.read().split()

next_bday = datetime.date.today()  + BDay(1)
index = 0
level_max_distance = 1.03

df_levels = pd.DataFrame(columns=['Date','symbol','support', 'sup_count','resistance', 'res_count'])
for sym in symbols:
    try:
        price, support, resistance = get_support_resistance(sym)
        #output only if support or resistance exist and less than 5 percent away from current price
        if (support[0] >0 and support[1] >= 10 and price/support[0] < level_max_distance) \
            or (resistance[0] >0 and resistance[1] >= 10 and resistance[0]/price < level_max_distance): 
            df_levels.loc[index] = [next_bday.strftime('%Y-%m-%d'), sym, support[0], support[1], resistance[0], resistance[1]]
            index = index+1
    except Exception:
        pass
df_levels.to_csv('sup-res-levels.txt')
print("processing completed")



167.07
39.28
60.92
112.49
33.66
92.88
41.97
154.14
11.03
112.02
68.7
168.95
40.31
processing completed
